In [22]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import statsmodels.api as sm

warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.utils import shuffle
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import optuna
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
import json
import joblib

# Setting the working directory
# os.chdir(r'../')
if os.getcwd().endswith('notebook'):
    os.chdir(r'../')
print('current working directory:', os.getcwd())

# Pandas option:
pd.set_option('max_colwidth',40)  # This allows us to see the data upto 40 characters per each column

pd.options.display.max_columns = None   # This allows us to see all the columns without collapsing 


# The below code is for for showing all expressions, not just the result of the last expression
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Table styles, this below code is for decorating the dataframe no functional use.
table_styles = {
        'cerulean_palette' : [
        dict(selector="th", props=[("color", "#FFFFFF"), ("background", "#004D80"), ("text-transform", "capitalize")]),
        dict(selector="td", props=[("color", "#333333")]),
        dict(selector="table", props=[("font-family", 'Arial'), ("border-collapse", "collapse")]),
        dict(selector='tr:nth-child(even)', props=[('background', '#D3EEFF')]),
        dict(selector='tr:nth-child(odd)', props=[('background', '#FFFFFF')]),
        dict(selector="th", props=[("border", "1px solid #0070BA")]),
        dict(selector="td", props=[("border", "1px solid #0070BA")]),
        dict(selector="tr:hover", props=[("background", "#80D0FF")])
    ]
}

current working directory: f:\NTU Learn\DS Thinking\ntu_sd6101_team_project


In [46]:
raw_data = pd.read_csv('../ntu_sd6101_team_project_data/zomato.csv', na_values=['-','',' ','NEW','[]'])
raw_data['id'] = raw_data.index
marked_data = pd.read_csv('../ntu_sd6101_team_project_data\subset_cyh\zomato_subset_cyh_labeled_10p.csv')
marked_data.rename(columns={'Unnamed: 0':'id'}, inplace=True)
location_df = pd.read_csv('../ntu_sd6101_team_project_data/location_info.csv')

In [24]:
d1 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/original_data_with_prediction.csv')
d2 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/new_data_with_predictions.csv')
d4 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/neated_ratings_have_marked_final11.csv')
d3 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/neated_ratings_need_mark_final11.csv')
d5 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/ratings_have_marked11.csv')
d6 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/ratings_need_mark11.csv')
d7 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/task1_training_samples_features_display.csv')
d8 = pd.read_csv('../ntu_sd6101_team_project_data/task1_data/task1_negative_samples_features_display.csv')

d1.shape, d2.shape, d3.shape, d4.shape, d5.shape, d6.shape, d7.shape, d8.shape

((23202, 13),
 (23202, 77),
 (23202, 76),
 (28288, 76),
 (28288, 12),
 (23202, 12),
 (28621, 120),
 (23096, 120))

In [25]:
d3.shape[0] + d4.shape[0], d7.shape[0]+d8.shape[0], raw_data.shape[0]

(51490, 51717, 51717)

In [ ]:
raw_data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),id
0,https://www.zomato.com/bangalore/jal...,"942, 21st Main Road, 2nd Stage, Bana...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, P...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful...",NaN,Buffet,Banashankari,0
1,https://www.zomato.com/bangalore/spi...,"2nd Floor, 80 Feet Road, Near Big Ba...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirva...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been he...",NaN,Buffet,Banashankari,1
2,https://www.zomato.com/SanchurroBang...,"1112, Next to KIMS Medical College, ...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is...",NaN,Buffet,Banashankari,2
3,https://www.zomato.com/bangalore/add...,"1st Floor, Annakuteera, 3rd Stage, B...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food ...",NaN,Buffet,Banashankari,3
4,https://www.zomato.com/bangalore/gra...,"10, 3rd Floor, Lakshmi Associates, G...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good r...",NaN,Buffet,Banashankari,4


In [26]:
def func1(string):
    l=string.split(", ")
    
    if "Afghan" in l:
        l=list(map(lambda x: x.replace('Afghan', 'Afghani'), l))
        
    if "Bubble Tea" in l:
        l=list(map(lambda x: x.replace('Bubble Tea', 'Beverages'), l))
        
    if "Coffee" in l:
        l=list(map(lambda x: x.replace('Coffee', 'Beverages'), l))
        
    if "Cafe" in l:
        l=list(map(lambda x: x.replace('Cafe', 'Beverages'), l))
       
    if "Tea" in l:
        l=list(map(lambda x: x.replace('Tea', 'Beverages'), l))
        
    if "Bubble Beverages" in l:
        l=list(map(lambda x: x.replace('Bubble Beverages', 'Beverages'), l))
       
    if "Ice Cream" in l:
        l=list(map(lambda x: x.replace('Ice Cream', 'Desserts'), l))
        
    if "Mithai" in l:
        l=list(map(lambda x: x.replace('Mithai', 'Desserts'), l))
      
    if "Bar Food" in l:
        l=list(map(lambda x: x.replace('Bar Food', 'Fast Food'), l))
        
    if "Burger" in l:
        l=list(map(lambda x: x.replace('Burger', 'Fast Food'), l))
       
    if "Finger Food" in l:
        l=list(map(lambda x: x.replace('Finger Food', 'Fast Food'), l))
        
    if "Momos" in l:
        l=list(map(lambda x: x.replace('Momos', 'Fast Food'), l))
        
    if "Rolls" in l:
        l=list(map(lambda x: x.replace('Rolls', 'Fast Food'), l))
    
    if "Wraps" in l:
        l=list(map(lambda x: x.replace('Wraps', 'Fast Food'), l))
        
    if "Street Food" in l:
        l=list(map(lambda x: x.replace('Street Food', 'Fast Food'), l))
 
    if "Juices" in l:
        l=list(map(lambda x: x.replace('Juices', 'Healthy Food'), l))
       
    if "Salad" in l:
        l=list(map(lambda x: x.replace('Salad', 'Healthy Food'), l))
     
    if "Sandwich" in l:
        l=list(map(lambda x: x.replace('Sandwich', 'Healthy Food'), l))
      
    if "Grill" in l:
        l=list(map(lambda x: x.replace('Grill', 'BBQ'), l))
    
    if "Steak" in l:
        l=list(map(lambda x: x.replace('Steak', 'BBQ'), l))
        
    if "Sushi" in l:
        l=list(map(lambda x: x.replace('Sushi', 'Japanese'), l))
        
    if "Tex-Mex" in l:
        l=list(map(lambda x: x.replace('Tex-Mex', 'Mexican'), l))
        
    if "Roast Chicken" in l:
        l=list(map(lambda x: x.replace('Roast Chicken', 'Chinese'), l))
        
    if "Charcoal Chicken" in l:
        l=list(map(lambda x: x.replace('Charcoal Chicken', 'Chinese'), l))
        
    if "Pizza" in l:
        l=list(map(lambda x: x.replace('Pizza', 'Italian'), l))
        
    if "Biryani" in l:
        l=list(map(lambda x: x.replace('Biryani', 'South Indian'), l))
        
    if "Kebab" in l:
        l=list(map(lambda x: x.replace('Kebab', 'North Indian'), l))
    
    return ", ".join(set(l))

In [27]:
def process_features(input_df):
    sample_df = input_df.copy()
    df = sample_df.drop([ 'url','phone','reviews_list','rest_type'],axis=1)

    df['cuisines'].fillna('Other', inplace=True)
    df["cuisines"] = df["cuisines"].apply(lambda x: func1(str(x)))
    df['votes'] = df['votes'].astype('int')
    df['approx_cost(for two people)'] = df['approx_cost(for two people)'].str.replace(",","").astype(float)
    df[-df['rate'].isna()]['rate'] = df[-df['rate'].isna()]['rate'].str.replace("/5","").astype(float)
    col = ["listed_in(city)","listed_in(type)"]
    ser = df.groupby(col)['approx_cost(for two people)'].transform('median')
    df['approx_cost(for two people)'] = df['approx_cost(for two people)'].fillna(ser)
    df['Cost_Per_Person'] = df['approx_cost(for two people)']/2
    df.drop(columns='approx_cost(for two people)', inplace=True)

    def p_category(price):
        if price<=100.0:
            return "Cheap"
        elif price<=250.0:
            return "Resonable"
        elif price<=500.0:
            return "Affordable"
        else:
            return "Expensive"

    df["rate"] = df["rate"].str[0:3].astype(float)
    df["Price_Category"] = df["Cost_Per_Person"].apply(p_category)

    # whether menu is available or not

    df['menu_item'].fillna('No',inplace=True)
    df["Menu"] = df["menu_item"].map(lambda x: "Yes" if x != 'No' else "No")

    #no of dishes liked by customer at a restaurant
    df[df['dish_liked'].isnull()]['dish_liked'] = ''
    df['dish_liked'] = df['dish_liked'].map(lambda x:0 if x=="" else len(str(x).split(", ")))

    # No of varieties served at a restaurant

    df['No_of_Varieties']=df['cuisines'].apply(lambda x:len(str(x).split(", ")))
    change = {'name':'Name',
            'address':'Address',
            'online_order':"Delivery",
            'book_table':"Booking", 
            'rate':"Rating", 
            'dish_liked':"No_of_Best_Sellers",
            'cuisines':"Cuisines", 
            'approx_cost(for two people)':"Average_Cost", 
            'listed_in(type)':"Type",
            'listed_in(city)':"City"}

    df.rename(columns=change,inplace=True)
    df.reset_index(drop=True,inplace=True)

    display_df=df[['id','Name','Address','Menu','Delivery', 'Booking', 'No_of_Best_Sellers', 'No_of_Varieties', 'Cuisines',
        'Cost_Per_Person', 'Type', 'City', 'Rating','Price_Category']]

    multi_label = display_df["Cuisines"].str.split(", ")
    mlb = MultiLabelBinarizer()
    inter_data = mlb.fit_transform(multi_label)
    multi_label_df = pd.DataFrame(inter_data,columns=mlb.classes_)
    display = pd.concat([display_df,multi_label_df],axis=1)

    one_hot = display[['Delivery', 'Booking', 'City']]
    numeric = display[['id','No_of_Best_Sellers', 'No_of_Varieties', 'Cost_Per_Person', 'Rating']]

    encoder = OneHotEncoder()
    one_hot_df = pd.DataFrame(encoder.fit_transform(one_hot).toarray(),index=list(display.index))

    df_inter = pd.concat([numeric,multi_label_df.iloc[list(display.index),:],one_hot_df],axis=1)
    df_inter.reset_index(drop=True,inplace=True)
    df_inter.head()

    return df_inter

In [28]:
features_df = process_features(raw_data)
features_df.head()

,id,No_of_Best_Sellers,No_of_Varieties,Cost_Per_Person,Rating,Afghani,African,American,Andhra,Arabian,Asian,Assamese,Australian,Awadhi,BBQ,Bakery,Belgian,Bengali,Beverages,Bihari,Bohri,British,Burmese,Cantonese,Chettinad,Chinese,Continental,Desserts,Drinks Only,European,Fast Food,French,German,Goan,Greek,Gujarati,Healthy Food,Hot dogs,Hyderabadi,Indian,Indonesian,Iranian,Italian,Japanese,Jewish,Kashmiri,Kerala,Konkan,Korean,Lebanese,Lucknowi,Maharashtrian,Malaysian,Malwani,Mangalorean,Mediterranean,Mexican,Middle Eastern,Modern Indian,Mongolian,Mughlai,Naga,Nepalese,North Eastern,North Indian,Oriya,Other,Paan,Pan Asian,Parsi,Portuguese,Rajasthani,Raw Meats,Russian,Seafood,Sindhi,Singaporean,South American,South Indian,Spanish,Sri Lankan,Tamil,Thai,Tibetan,Turkish,Vegan,Vietnamese,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0,7,3,400.0,4.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7,3,400.0,4.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,7,3,400.0,3.8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1,2,150.0,3.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2,2,300.0,3.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
def calculate_customer_preference(marks):
    preferences = []
    for mark in marks:
        preferece = mark/5
        preferences.append(preferece)
    
    user_preference = np.mean(preferences)

    return user_preference

In [49]:
marked_data['customer_preference'] = 0

for i in range(marked_data.shape[0]):

    marks = marked_data.iloc[i, -10:].values.tolist()

    marked_data.loc[i, 'customer_preference'] = calculate_customer_preference(marks)

marked_data.head()

,id,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),Rajesh Naik,Megha Reddy,Vinod Sharma,Anjali Singh,Ali Khan,Sneha Iyer,Manoj Desai,Gopal Gupta,Neha Verma,Ashok Reddy,customer_preference
0,8440,https://www.zomato.com/bangalore/kal...,"399, 16th Main, N.S Palya, BTM, Bang...",Kalingas,Yes,No,3.9,179,080 42038180,BTM,Quick Bites,"Rasgulla, Mutton Kosha, Chicken Kash...","Oriya, Fast Food",250,"[('Rated 3.0', 'RATED\n A pocket fr...",NaN,Delivery,BTM,4,4,4,4,5,4,4,4,4,5,0.76
1,23575,https://www.zomato.com/bangalore/ang...,"1st Cross, Bilekahalli, Near IIMB Co...",Angel Restaurant,Yes,No,2.8,165,+91 9035699847\r\n+91 9880714970,Bannerghatta Road,Quick Bites,NaN,"Kerala, South Indian, North Indian",450,"[('Rated 1.0', 'RATED\n I cannot sa...","['Kappa', 'Kappa Biryani', 'Chicken ...",Dine-out,JP Nagar,3,3,4,3,3,4,3,3,3,4,0.60
2,16711,https://www.zomato.com/bangalore/caf...,"4rd floor, 7th Cross, Opposite to BM...",Cafe Talkhouse,No,No,3.8,23,NaN,HSR,Quick Bites,NaN,North Indian,NaN,"[('Rated 5.0', 'RATED\n Have been t...",NaN,Delivery,HSR,4,4,4,4,4,4,4,4,4,5,0.74
3,42588,https://www.zomato.com/bangalore/par...,"17/1 Cambridge Road, Opposite The Fr...",Paratha Xpress,No,No,3.5,8,065535 07908402366,Ulsoor,Quick Bites,NaN,"North Indian, Chinese",200,"[('Rated 5.0', 'RATED\n good job!')...",NaN,Delivery,MG Road,4,4,4,4,4,4,4,4,4,5,0.74
4,5266,https://www.zomato.com/bangalore/cla...,"Skywalk, 5/1, Near Body Craft, Assay...",Classic Lassi Shop,Yes,No,3.5,43,+91 8792958786\r\r\n+91 8073579529,Ulsoor,Beverage Shop,NaN,"Beverages, Ice Cream",150,"[('Rated 3.0', 'RATED\n I have mixe...","['Fruit Salad with Ice Cream', 'Brow...",Delivery,Brigade Road,4,4,4,4,4,4,4,4,4,4,0.72


In [50]:
all_data = pd.merge(raw_data, features_df, on='id', how='left')
all_data = pd.merge(all_data, marked_data[['id', 'customer_preference']], on='id', how='left')
all_data = pd.merge(all_data, location_df, on='location', how='left')
all_data['rate'] = all_data['rate'].apply(lambda x: float(x.split('/')[0]) if type(x) == str else x)
all_data['rate'] = all_data['rate'].fillna(all_data['Rating'].mean())
all_data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),id,No_of_Best_Sellers,No_of_Varieties,Cost_Per_Person,Rating,Afghani,African,American,Andhra,Arabian,Asian,Assamese,Australian,Awadhi,BBQ,Bakery,Belgian,Bengali,Beverages,Bihari,Bohri,British,Burmese,Cantonese,Chettinad,Chinese,Continental,Desserts,Drinks Only,European,Fast Food,French,German,Goan,Greek,Gujarati,Healthy Food,Hot dogs,Hyderabadi,Indian,Indonesian,Iranian,Italian,Japanese,Jewish,Kashmiri,Kerala,Konkan,Korean,Lebanese,Lucknowi,Maharashtrian,Malaysian,Malwani,Mangalorean,Mediterranean,Mexican,Middle Eastern,Modern Indian,Mongolian,Mughlai,Naga,Nepalese,North Eastern,North Indian,Oriya,Other,Paan,Pan Asian,Parsi,Portuguese,Rajasthani,Raw Meats,Russian,Seafood,Sindhi,Singaporean,South American,South Indian,Spanish,Sri Lankan,Tamil,Thai,Tibetan,Turkish,Vegan,Vietnamese,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,customer_preference,location_id,restaurant_count,latitude,longitude,northeast_lat,northeast_lng,southwest_lat,southwest_lng,area,area_proportion,area_population,area_customer_flow
0,https://www.zomato.com/bangalore/jal...,"942, 21st Main Road, 2nd Stage, Bana...",Jalsa,Yes,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, P...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful...",NaN,Buffet,Banashankari,0,7,3,400.0,4.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,906.0,12.925453,77.546757,12.956544,77.574414,12.912888,77.531272,2.260970e+07,0.002921,1.083382e+06,3164.3497
1,https://www.zomato.com/bangalore/spi...,"2nd Floor, 80 Feet Road, Near Big Ba...",Spice Elephant,Yes,No,4.1,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirva...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been he...",NaN,Buffet,Banashankari,1,7,3,400.0,4.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,906.0,12.925453,77.546757,12.956544,77.574414,12.912888,77.531272,2.260970e+07,0.002921,1.083382e+06,3164.3497
2,https://www.zomato.com/SanchurroBang...,"1112, Next to KIMS Medical College, ...",San Churro Cafe,Yes,No,3.8,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is...",NaN,Buffet,Banashankari,2,7,3,400.0,3.8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,906.0,12.925453,77.546757,12.956544,77.574414,12.912888,77.531272,2.260970e+07,0.002921,1.083382e+06,3164.3497
3,https://www.zomato.com/bangalore/add...,"1st Floor, Annakuteera, 3rd Stage, B...",Addhuri Udupi Bhojana,No,No,3.7,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food ...",NaN,Buffet,Banashankari,3,1,2,150.0,3.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [52]:
ht_data = all_data[all_data['id'].isin(marked_data['id'].values.tolist())]
ht_data.shape

(1000, 151)

In [63]:
# Model: customer_preference ~ rate
# H0: customer_preference dosen't hava significant relationship with rate
# H1: customer_preference has significant relationship with rate

X1 = ht_data.loc[:,['rate']]
y1 = ht_data['customer_preference']

X1 = sm.add_constant(X1)
glm_model_1 = sm.OLS(y1, X1)
glm_results_1 = glm_model_1.fit()

print(glm_results_1.summary())

                             OLS Regression Results                            
Dep. Variable:     customer_preference   R-squared:                       0.036
Model:                             OLS   Adj. R-squared:                  0.035
Method:                  Least Squares   F-statistic:                     36.74
Date:                 Wed, 06 Nov 2024   Prob (F-statistic):           1.91e-09
Time:                         13:52:42   Log-Likelihood:                 105.68
No. Observations:                 1000   AIC:                            -207.4
Df Residuals:                      998   BIC:                            -197.5
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2139      0.069      3.10

In [ ]:
# Model: customer_preference ~ restaurant_count
# H0: customer_preference dosen't hava significant relationship with restaurant_count
# H1: customer_preference has significant relationship with restaurant_count

X1 = ht_data.loc[:,['restaurant_count']]
y1 = ht_data['customer_preference']

X1 = sm.add_constant(X1)
glm_model_1 = sm.OLS(y1, X1)
glm_results_1 = glm_model_1.fit()

print(glm_results_1.summary())

                             OLS Regression Results                            
Dep. Variable:     customer_preference   R-squared:                       0.000
Model:                             OLS   Adj. R-squared:                 -0.001
Method:                  Least Squares   F-statistic:                    0.4094
Date:                 Wed, 06 Nov 2024   Prob (F-statistic):              0.522
Time:                         13:53:34   Log-Likelihood:                 87.807
No. Observations:                 1000   AIC:                            -171.6
Df Residuals:                      998   BIC:                            -161.8
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6246   